In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [29]:
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
train.shape

In [30]:
Target = 'SalePrice'
train.dropna(axis=0, subset=[Target], inplace=True)


In [31]:
train.shape

In [32]:
train = train.drop(columns=['Id'],axis=1)


In [33]:
import missingno as msno
msno.matrix(train)


In [34]:
train.columns[train.isnull().any()]
train_nan_95 = train.columns[train.isnull().any()]

In [35]:
for column in train_nan_95:
    if train[column].isnull().sum()*100.0/train.shape[0] > 60:
        train.drop(column,1, inplace=True)

In [36]:
print(train.shape)
train.columns[train.isnull().any()]

In [37]:
train = train.drop(['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual','SaleType'], axis=1)
train.columns[train.isnull().any()]

In [38]:
train['GarageCond'] = train.fillna(train['GarageCond'].value_counts().index[0])

mean_value=train['LotFrontage'].mean()
train['LotFrontage'].fillna(value=mean_value, inplace=True)

In [39]:
def getObjectColumnsList(train):
    return [cname for cname in train.columns if train[cname].dtype == "object"]

def PerformOneHotEncoding(train,columnsToEncode):
    return pd.get_dummies(train,columns = columnsToEncode)

cat_cols = getObjectColumnsList(train)
train = PerformOneHotEncoding(train,cat_cols)
train.head()

In [40]:
train_data=train.iloc[:1460,:]
print(train_data.shape)


In [41]:
X=train_data
Y=train.loc[:,'SalePrice']

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2)

In [43]:
import xgboost as xgb

model_xgb = xgb.XGBRegressor(n_estimators=340, max_depth=2, learning_rate=0.2)
model_xgb.fit(X_train, Y_train)
xgb_preds=model_xgb.predict(X_test)

In [51]:
from sklearn import metrics

score_1 = metrics.r2_score(Y_test, xgb_preds)
print("R squared error : ", score_1)